In [18]:
import pandas as pd
import numpy as np
import nltk
import string
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense


#tokenize the titles
def tokenizeTitles(data):
    tokens = [] #array that keeps tokenized titles
    for i in range(len(data)):
        tokens.append((nltk.tokenize.word_tokenize(data[i]))) 
    return tokens

#Stem Function
def stemming(tokens):
    stemmer = nltk.stem.PorterStemmer()
    filtered_data = [] #array that keeps stemmed titles
    for text in tokens:
        for word in text:
            filtered_data.append(stemmer.stem(word)) #filtrers data with stemming them
    return filtered_data

#stopwords removal function
def stopwordsRemoval(data): 
    stop_words = set(nltk.corpus.stopwords.words("english"))
    filtered_data= [] #array that keeps filtered data without stopwords
    for word in data:
        if not word in stop_words:
            filtered_data.append(w)
    return filtered_data

def removePunctuations(data):
    punctuations = set(string.punctuation)
    not_punctuation_data = [word for word in data if word not in punctuations] #keeps words that are not puncuations
    return not_punctuation_data

#tf-idf function
def setTfIdf(cleaned_data):
    tfidf = TfidfVectorizer(tokenizer=tokenizer, lowercase=False)
    weighted_data = tfidf.fit_transform(cleaned_data).toarray() #puts weight on data
    return weighted_data

def tokenizer(text):
    return text    

#preprocess data function
def dataPreprocessing(data):
    cleaned_data = tokenizeTitles(data)
    cleaned_data = stemming(cleaned_data)
    cleaned_data = stopwordsRemoval(cleaned_data)
    cleaned_data = removePunctuations(cleaned_data)
    return cleaned_data
    
def main():
    data = pd.read_csv('onion-or-not.csv')
    df = pd.DataFrame(data)
    data = df['text']
    
    #data preprocessing
    cleaned_data = dataPreprocessing(data)
    
    #training/test arrays
    X = setTfIdf(cleaned_data)
    y = np.array(df['label']).transpose()
    
    #creation of keras model
    model = Sequential()
    model.add(Dense(12, input_shape=(247715,), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    # compile the keras model 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

    # fit the keras model on the dataset
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25)
    model.fit(X_train, Y_train, epochs=150, batch_size=10)

    # evaluate the keras model
    accuracy = model.evaluate(X_test, Y_test)
    print('Accuracy: %.2f' % (accuracy*100))
      
if __name__ == "__main__":
    main()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 12)                2972592   
_________________________________________________________________
dense_26 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 9         
Total params: 2,972,705
Trainable params: 2,972,705
Non-trainable params: 0
_________________________________________________________________


ValueError: Found input variables with inconsistent numbers of samples: [247715, 24000]